In [59]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import re
import time

In [60]:
car_classifier = cv2.CascadeClassifier('DATA/haarcascades/haarcascade_car.xml')

In [61]:
def detect_car(img):
    car_img = img.copy()
    car_img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    car_rects = car_classifier.detectMultiScale(car_img_gray)
    
    for(x,y,w,h) in car_rects:
        cv2.rectangle(car_img,(x,y),(x+w,y+h),(0,0,255),2)
    return car_img

In [62]:
response = requests.get('https://digilive.rcs-rds.ro/digilivedge/satu_mare_desktop.stream/index.m3u8')
match = re.findall(r'segment_(\d+)\.ts', response.text)
last_segment = match[-1]


while True:

    cap = cv2.VideoCapture('https://digilive.rcs-rds.ro/digilivedge/satu_mare_desktop.stream/segment_' + last_segment + '.ts')

    if cap.isOpened() == False:
        print('ERROR FILE NOT FOUND OR WRONG, CODEC USED!')

    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            result = detect_car(frame)
            cv2.imshow('frame', result)
            time.sleep(1/20)
            if cv2.waitKey(1) & 0xFF == 27:
                break
        else:
            break
    incremented_number = int(last_segment) + 1
    last_segment = f"{incremented_number:0{len(last_segment)}d}"
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()                      